In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2020-11-14 17:19:55--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2020-11-14 17:19:56 (8.92 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [6]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.crnebbbwgpbv.us-east-2.rds.amazonaws.com:5432/Challenge"
config = {"user":"postgres",
          "password": "password",
          "driver":"org.postgresql.Driver"}

In [7]:
vine_table_df = spark.read.jdbc(url=jdbc_url, table="vine_table", properties=config)

In [8]:
vine_table_df = vine_table_df.filter("total_votes >= 20")
vine_table_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R9KUD77KL30OW|          1|           34|         37|   N|                Y|
| RT9FA5RTGR227|          3|           18|         21|   N|                Y|
| R75E5C0D4B4BO|          5|           96|         96|   N|                Y|
|R2W2XRKRI6F34M|          4|           41|         44|   N|                Y|
|R16UIXY20Z1KEN|          1|           43|         58|   N|                Y|
|R26MG5HTIQ3THC|          1|           12|         27|   N|                N|
|R1YAS14SEOHCKR|          1|           26|         27|   N|                Y|
|R3BB7YXWNRR7BQ|          3|           19|         23|   N|                Y|
|R1QFAGDAVZX7CC|          3|           40|         47|   N|                N|
|R1L2Q87GINYW9O|          5|           68|         69|   N|     

In [9]:
vine_df2 = vine_table_df.filter(vine_table_df["total_votes"]>=20)
vine_df3 = vine_df2.filter(vine_df2["helpful_votes"]/vine_df2["total_votes"]>=0.5)

In [10]:
vine_df3.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R9KUD77KL30OW|          1|           34|         37|   N|                Y|
| RT9FA5RTGR227|          3|           18|         21|   N|                Y|
| R75E5C0D4B4BO|          5|           96|         96|   N|                Y|
|R2W2XRKRI6F34M|          4|           41|         44|   N|                Y|
|R16UIXY20Z1KEN|          1|           43|         58|   N|                Y|
|R1YAS14SEOHCKR|          1|           26|         27|   N|                Y|
|R3BB7YXWNRR7BQ|          3|           19|         23|   N|                Y|
|R1QFAGDAVZX7CC|          3|           40|         47|   N|                N|
|R1L2Q87GINYW9O|          5|           68|         69|   N|                Y|
|R28XPXOAKTOGDC|          1|           34|         39|   N|     

In [11]:
vine_paid_df = vine_df3.filter(vine_df3["vine"]=="Y")

In [12]:
vine_paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| REIVEVPMTQN6I|          3|           36|         45|   Y|                N|
| R439N6WDCV4IF|          3|           59|         68|   Y|                N|
|R3NDAQ2XAEKM0J|          1|           33|         40|   Y|                N|
| R9L1TDFPFXBOU|          4|           29|         30|   Y|                N|
|R1S3N2W0P3KJ2Z|          3|           46|         47|   Y|                N|
| R2QWFFNM5CZHD|          5|          257|        264|   Y|                N|
|R1IGUULSJIS9Q6|          3|           21|         26|   Y|                N|
|R2WN72XKRQ8D1U|          5|           87|         91|   Y|                N|
|R1DJPG0OL6OP4H|          3|          414|        491|   Y|                N|
|R1TR9RQXUVEOXT|          3|          163|        170|   Y|     

In [13]:
vine_notpaid_df = vine_df3.filter(vine_df3["vine"]=="N")

In [14]:
vine_notpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R9KUD77KL30OW|          1|           34|         37|   N|                Y|
| RT9FA5RTGR227|          3|           18|         21|   N|                Y|
| R75E5C0D4B4BO|          5|           96|         96|   N|                Y|
|R2W2XRKRI6F34M|          4|           41|         44|   N|                Y|
|R16UIXY20Z1KEN|          1|           43|         58|   N|                Y|
|R1YAS14SEOHCKR|          1|           26|         27|   N|                Y|
|R3BB7YXWNRR7BQ|          3|           19|         23|   N|                Y|
|R1QFAGDAVZX7CC|          3|           40|         47|   N|                N|
|R1L2Q87GINYW9O|          5|           68|         69|   N|                Y|
|R28XPXOAKTOGDC|          1|           34|         39|   N|     

In [75]:
# Total number of paid 5 star review
paid_df = vine_paid_df.filter(vine_paid_df["star_rating"]=="5")
paid_df.count()

139

In [16]:
# Total number of not paid 5 star review
notpaid_df = vine_notpaid_df.filter(vine_notpaid_df["star_rating"]=="5")
notpaid_df.count()

32665

In [24]:
from pyspark.sql.functions import count

In [89]:
# Table with star_rating total of paid
percentage_df = vine_paid_df.groupby("star_rating").agg(count("star_rating")).withColumnRenamed("count(star_rating)","total")
percentage_paid_df = percentage_df.orderBy(percentage_df["star_rating"].desc())
percentage_paid_df.show()

+-----------+-----+
|star_rating|total|
+-----------+-----+
|          5|  139|
|          4|  118|
|          3|   56|
|          2|   13|
|          1|    8|
+-----------+-----+



In [91]:
# Total count of paid 5 star ratings review 
y = vine_paid_df.count()
print(y)

334


In [90]:
# Percentage of paid 5 star reviews
percentage_paid_df.withColumn("percentage", percentage_df["total"]/y *100).show()

+-----------+-----+------------------+
|star_rating|total|        percentage|
+-----------+-----+------------------+
|          5|  139| 41.61676646706587|
|          4|  118| 35.32934131736527|
|          3|   56|16.766467065868262|
|          2|   13|3.8922155688622757|
|          1|    8|2.3952095808383236|
+-----------+-----+------------------+



In [92]:
# Table with star_rating total of not paid
percentage_df2 = vine_notpaid_df.groupby("star_rating").agg(count("star_rating")).withColumnRenamed("count(star_rating)","total")
percentage_notpaid_df = percentage_df2.orderBy(percentage_df["star_rating"].desc())
percentage_notpaid_df.show()

+-----------+-----+
|star_rating|total|
+-----------+-----+
|          5|32665|
|          4|10688|
|          3| 5477|
|          2| 3592|
|          1| 9192|
+-----------+-----+



In [93]:
# Total count of not paid 5 star ratings review 
x = vine_notpaid_df.count()
print(x)

61614


In [95]:
# Percentage of not paid 5 star reviews
percentage_notpaid_df.withColumn("percentage", percentage_notpaid_df["total"]/x *100).show()

+-----------+-----+------------------+
|star_rating|total|        percentage|
+-----------+-----+------------------+
|          5|32665| 53.01554841432142|
|          4|10688|17.346706917259063|
|          3| 5477| 8.889213490440484|
|          2| 3592|5.8298438666536825|
|          1| 9192|14.918687311325348|
+-----------+-----+------------------+

